In [4]:
import numpy as np
import pandas as pd
from scipy.signal import correlate
import matplotlib.pyplot as plt
from scipy.sparse import lil_matrix
from itertools import product
from itertools import combinations
from datetime import datetime, timedelta
from multiprocessing import Pool


# Load the temporal series data
data_with_date = pd.read_csv('temporal_series_32.csv')
data = data_with_date.drop(columns=['date'])

#Inizialize parameters
max_tau = 15 
threshold = 0.05

#Function to compute maximum of time correlation function between two temporal series
def time_delayed_cross_correlation(df, col1, col2, max_tau, threshold=0.05):
    """
    Optimized custom time-delayed cross-correlation function for two time series.

    Args:
    df (pandas.DataFrame): Dataframe containing the time series data.
    col1 (str): Name of the first time series column.
    col2 (str): Name of the second time series column.
    max_tau (int): Maximum time delay (positive or negative).
    threshold (float): Threshold for maximum correlation.

    Returns:
    float: Maximum cross-correlation above the threshold, or 0 if none.
    """
    # Extract the series and compute means and std deviations
    series_i = df[col1].values
    series_j = df[col2].values
    mean_i, std_i = series_i.mean(), series_i.std()
    mean_j, std_j = series_j.mean(), series_j.std()
    
    # Check if standard deviations are zero to avoid division by zero
    if std_i == 0 or std_j == 0:
        return 0
    
    # Initialize array to store cross-correlations for each lag
    cross_corrs = np.zeros(2 * max_tau + 1)

    # Compute cross-correlation for each lag
    for tau in range(-max_tau, max_tau + 1):
        if tau < 0:
            # Positive lag: shift series_j forward by -tau (series_i aligns with delayed series_j)
            numerator = np.sum((series_i[-tau:] - mean_i) * (series_j[:len(series_j) + tau] - mean_j))
            denominator = (std_i * std_j * (len(series_i) + tau))
        else:
            # Negative or zero lag: shift series_i forward by tau (series_j aligns with delayed series_i)
            numerator = np.sum((series_i[:len(series_i) - tau] - mean_i) * (series_j[tau:] - mean_j))
            denominator = (std_i * std_j * (len(series_i) - tau))

        # Calculate cross-correlation, check for zero denominator
        cross_corrs[tau + max_tau] = numerator / denominator if denominator != 0 else 0

    # Find the maximum correlation and compare with threshold
    max_corr = np.max(cross_corrs)
    max_index = np.argmax(cross_corrs)
    best_tau = max_index - max_tau  # Convert array index back to tau value

    # Apply threshold check and adjust max_corr based on tau direction
    if max_corr >= threshold:
        return max_corr if best_tau >= 0 else -max_corr
    else:
        return 0


#Function to compute the whole adjacency matrix (impossible to compute)
def ccr_matrix(df):
    #build the adjacency matrix
    adjacency_matrix = pd.DataFrame(np.nan, index=df.columns, columns=df.columns)

    #iterate for each pair of columns
    for col1 in df.columns:
        for col2 in df.columns:
            adjacency_matrix.loc[col1,col2] = time_delayed_cross_correlation(df, col1, col2, max_tau)

    return adjacency_matrix

#function to compute just a submatrix of the adjacency matrix (submatricese on the diagonal)
def compute_submatrix(df, columns, filename, max_tau):
    submatrix = pd.DataFrame(index=columns, columns=columns)

    # Loop through column pairs, skipping redundant calculations
    for col1, col2 in product(columns, repeat=2):
        if col1 <= col2:  # Ensures each pair is calculated only once
            correlation = time_delayed_cross_correlation(df, col1, col2, max_tau)
            submatrix.loc[col1, col2] = correlation
            submatrix.loc[col2, col1] = correlation  # Fill symmetric position

    submatrix.to_csv(filename)
    print(f"Submatrix {filename} has been saved.")
    return submatrix

#function to computer a submatrix which is not on the diagonal
def compute_cross_group_matrix(df, group_a, group_b, filename):
    cross_matrix = pd.DataFrame(index=group_a, columns=group_b)
    for col1, col2 in product(group_a, group_b):
        cross_matrix.loc[col1, col2] = time_delayed_cross_correlation(df, col1, col2, max_tau)
    cross_matrix.to_csv(filename)
    print(f"submatrix {filename} has been saved")
    return cross_matrix

#Now to compute the submatrices we need to divide the 3104 columns into groups
def divide_into_groups(df, group_sizes):
    #Divides the comlumns into specified sizes
    columns = df.columns.tolist()
    groups = []
    start = 0
    for size in group_sizes:
        groups.append(columns[start:start + size])
        start += size
    return groups

#Change of program, now we will try to compute manually the single pieces
group_1 = data.columns[0:1000]
group_2 = data.columns[1000:2000]
group_3 = data.columns[2000:3104]


filename_sub_1 = "32_sub_mat_1.csv"
compute_submatrix(data, group_1, filename_sub_1, max_tau)

filename_sub_2 = "32_sub_mat_2.csv"
compute_submatrix(data, group_2, filename_sub_2, max_tau)

filename_sub_3 = "32_sub_mat_3.csv"
compute_submatrix(data, group_3, filename_sub_3, max_tau)

filename_cross_1 = "32_cross_matrix_1_2.csv"
compute_cross_group_matrix(data, group_1, group_2, filename_cross_1)

filename_cross_2 = "32_cross_matrix_1_3.csv"
compute_cross_group_matrix(data, group_1, group_3, filename_cross_2)

filename_cross_3 = "32_cross_matrix_2_3.csv"
compute_cross_group_matrix(data, group_2, group_3, filename_cross_3)


Submatrix 32_sub_mat_1.csv has been saved.
Submatrix 32_sub_mat_2.csv has been saved.
Submatrix 32_sub_mat_3.csv has been saved.
submatrix 32_cross_matrix_1_2.csv has been saved
submatrix 32_cross_matrix_1_3.csv has been saved
submatrix 32_cross_matrix_2_3.csv has been saved


38099.0   38101.0   38103.0   38105.0   39001.0   39003.0   39005.0  \
21083.0   0.43205  0.675296 -0.621752 -0.658451  0.642372  0.659594 -0.668271   
21085.0 -0.498843 -0.779522 -0.804686  0.716531  0.718563  0.683733  0.721948   
21087.0  -0.56464   0.74486 -0.780001 -0.642477  0.678533  0.660524  0.717752   
21089.0 -0.552743  0.730594 -0.706599  0.668634   0.69239  0.666578   -0.7267   
21091.0  0.524293  0.705363 -0.614248 -0.749409  0.727639  0.725865  0.724123   
...           ...       ...       ...       ...       ...       ...       ...   
38089.0  0.616076  0.923049 -0.801922  0.910667  0.729985  0.823412  0.793103   
38091.0  0.625934  0.743463 -0.583285 -0.701393 -0.642763 -0.707284  0.652257   
38093.0  0.839007  0.895905  0.738497  0.885189  0.770898  0.808638  0.789541   
38095.0  0.591159  0.749619 -0.706395  0.739217  0.712877 -0.746513 -0.762094   
38097.0  0.821169  0.831892 -0.652663  0.833424  0.596396  0.702415 -0.661177   

          39007.0   39009.0   39011.0  ...   56027.0   56029.0   56031.0  \
21083.0  0.669145  0.660999  0.677326  ...  0.440888  0.657818  0.551365   
21085.0  0.771381  0.715893  0.711594  ...  0.423012  0.806821 -0.688666   
21087.0  0.805508  0.703999  0.701543  ... -0.335153  0.812677   0.76543   
21089.0  0.789902 -0.708392  0.720684  ... -0.349056  0.776719  0.729808   
21091.0  0.682719  0.688598  0.732601  ... -0.533269   0.70049  0.655957   
...           ...       ...       ...  ...       ...       ...       ...   
38089.0   0.81093  0.745679  0.827101  ...  0.490393  0.846464 -0.800636   
38091.0 -0.702016  0.648335   0.71762  ...  0.557419 -0.684205 -0.725562   
38093.0  0.842636   0.76243  0.810766  ... -0.415915  0.838487  0.843509   
38095.0  0.720397  0.725304 -0.759729  ... -0.530473 -0.733883 -0.701719   
38097.0 -0.623346  0.623169  0.699078  ...  0.552569  -0.61142  0.608293   

          56033.0   56035.0   56037.0   56039.0   56041.0   56043.0   56045.0  
21083.0 -0.670665  0.582856 -0.668705   0.72186  0.681707 -0.221082  0.677311  
21085.0   0.77472  0.682209  0.777302  0.812105  0.764621 -0.180196  0.814901  
21087.0  0.714263  0.724016  0.640916  0.743596  0.703154 -0.153483  0.715805  
21089.0  0.677419  0.735578  0.595098   0.75201  0.747224 -0.179051  0.653414  
21091.0  0.740382  0.644774  0.725527  0.805614  0.792346 -0.235071 -0.696049  
...           ...       ...       ...       ...       ...       ...       ...  
38089.0  0.943697  0.709947  0.952257  0.904991  0.878927  0.225235  0.910476  
38091.0 -0.729603 -0.540986 -0.719289 -0.712681  -0.74266 -0.241561 -0.797184  
38093.0 -0.837593  0.691978 -0.730108  0.778486 -0.877819 -0.213446  0.764738  
38095.0 -0.713499 -0.623302 -0.697962 -0.709272 -0.785793 -0.261807 -0.773533  
38097.0 -0.706507 -0.586397 -0.710294 -0.730614 -0.721601  -0.29143  0.721729  

[1000 rows x 1104 columns]